In [14]:
%config IPCompleter.greedy=True

### Tensors

In [155]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x=torch.tensor([[1,1],[-1,-1]])
print(x)
y=torch.tensor([[0,0],[1,1]])
print(y)
print(torch.add(x,y))
print(y.add_(x))
x0 = x.view(4)
print(x0)
x=torch.empty(5,3)
z = torch.randn(4,4,4)
z0 = z.view(-1,8,2)
print(z0)
print(z0.size())
print(torch.cuda.is_available())

tensor([[ 1,  1],
        [-1, -1]])
tensor([[0, 0],
        [1, 1]])
tensor([[1, 1],
        [0, 0]])
tensor([[1, 1],
        [0, 0]])
tensor([ 1,  1, -1, -1])
tensor([[[ 1.8226e-01,  1.2252e+00],
         [ 1.1813e+00,  2.0014e+00],
         [ 1.9121e-01,  7.3609e-01],
         [-1.3047e+00, -1.1885e+00],
         [ 8.8603e-02, -4.2684e-01],
         [-8.4861e-01,  4.8832e-01],
         [-2.4969e+00,  1.8266e-03],
         [-5.8100e-01, -1.0611e-01]],

        [[-8.2451e-01,  1.7614e-01],
         [-1.5222e+00,  2.5711e-01],
         [ 3.7536e-01, -1.0872e+00],
         [-1.9636e-02, -2.2241e+00],
         [ 1.9197e+00,  1.0906e+00],
         [-6.2616e-01, -4.8243e-01],
         [ 1.9167e-01,  7.7307e-03],
         [-6.7898e-01,  8.9136e-01]],

        [[-3.2038e-01, -1.3194e+00],
         [ 5.5340e-01, -3.2076e-01],
         [ 7.6851e-01, -2.7730e-01],
         [-5.8879e-01,  6.2061e-01],
         [ 3.1452e-01,  1.9476e-01],
         [-5.7015e-01, -1.7932e+00],
         [ 6.0121e-01

###  AutoGrad

In [42]:
x = torch.rand(2,2,requires_grad=True)
#print(x)
y = x + 2
print(y.grad_fn)
z = y * y * 3
z_mean = z.mean()
print(z)
print(z_mean)
z_mean.backward()
print(x.grad)

tensor([[14.7668, 20.4317],
        [17.1685, 23.1552]], grad_fn=<MulBackward0>)
tensor(18.8806, grad_fn=<MeanBackward1>)
tensor([[3.3279, 3.9146],
        [3.5884, 4.1673]])


In [43]:
x = torch.randn(3,3,requires_grad=True)
print(x)

tensor([[-0.7367, -1.2929,  0.1704],
        [-0.4890, -0.2523,  0.0469],
        [-0.2459,  0.2969, -2.6363]], requires_grad=True)


### MNIST TASK

In [180]:
from torch.autograd import Variable
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(784,15)
        self.fc2 = nn.Linear(15,10)
        self.fc3 = nn.Linear(10,10)
        
    def forward(self,x):
        x=torch.sigmoid(self.fc1(x))
        x=torch.sigmoid(self.fc2(x))
        x=torch.sigmoid(self.fc3(x))
        return x
    
    
    def train(self, data_loader, epochs=10, bs=64, criterion=nn.CrossEntropyLoss()):
        opt = optim.SGD(self.parameters(), lr=0.01)
        for epoch in range(epochs):
            print("Epoch number=" + str(epoch))
            for xb,yb in data_loader:
                xb = xb.reshape(-1,28*28)
                pred = self(xb)
                print("Accuracy of model =" + str(accuracy(pred,yb)))
                #print(pred)
                #print(yb)
                loss = criterion(pred, yb)
                #print("loss=", loss)
                ###
                loss.backward()
                opt.step()
                opt.zero_grad() 
                
            
            
            


In [181]:
from pathlib import Path
import requests

# Extracting the data
DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
        
# Creating tensors instead of numpy arrays
x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid))
print(y_train)
print(y_valid)
print(y_valid.size())
# Creating Dataset
tr_data_set = TensorDataset(x_train, y_train)
tr_data_loader = DataLoader(tr_data_set, batch_size=64)



tensor([5, 0, 4,  ..., 8, 4, 8])
tensor([3, 8, 6,  ..., 5, 6, 8])
torch.Size([10000])


In [182]:
net=Net()
#print(loss_func(net(x_train),y_train))
print('test1')
net.train(tr_data_loader)
print('test')



test1
Epoch number=0
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1719)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.0469)
Accuracy of model =tensor(0

Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1719)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0469)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1406)
Accuracy of mo

Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1719)
Accuracy of model =tensor(0.2188)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1719)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1406)
Accuracy of mo

Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1719)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.0469)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1250)
Accuracy of mo

Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0469)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0469)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of mo

...

Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0469)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1719)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.0938)
Accuracy of mo

Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1719)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1406)
Accuracy of mo

Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1719)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.0312)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.2031)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0938)
Accuracy of mo

Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.2031)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0469)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0469)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1719)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1406)
Accuracy of mo

Accuracy of model =tensor(0.1562)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.0625)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1719)
Accuracy of model =tensor(0.1875)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.0781)
Accuracy of model =tensor(0.0938)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1406)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.1094)
Accuracy of model =tensor(0.1250)
Accuracy of model =tensor(0.0938)
Accuracy of mo